## Import Dependencies

In [1]:
import sys
import os
import time
import pyvisa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binascii import unhexlify
import serial

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\Drivers")
    
from Driver_Laser_Poet import *
from Driver_OSA_MS9740B import *

In [2]:
rm = pyvisa.ResourceManager()

osa = OSA_MS9740B()
osa.open(rm)

le = LE_POET()
le.open(rm)
le.set_timeout(10000)
rf = le.laser.query('*IDN?')
print(rf)

rf = le.set_fan_on(0) 
print(rf)
rf = le.set_fan_on(2) 
print(rf)

OSA Attenuation Status:  ON

POET,ELS_8Channel,POET20240116-01,V004
Set channel 0 ON
Set channel 2 ON


## Initialization

#### Turn off all LE Channels and set bias current to 0 mA

In [53]:
rf = le.set_output_off('0|1|2|3|4|5|6|7')
rf = le.get_optical_output_status('0|1|2|3|4|5|6|7')
print(rf)
rf = le.set_laser_current('0|1|2|3|4|5|6|7',0)
rf = le.get_laser_current('0|1|2|3|4|5|6|7')
print(rf)

0,OFF|1,OFF|2,OFF|3,OFF|4,OFF|5,OFF|6,OFF|7,OFF
0,0.00|1,0.00|2,0.00|3,0.00|4,0.00|5,0.00|6,0.00|7,0.00


#### Define measurement conditions

In [46]:
T_Start = 25
T_Stop = 50
T_Step = 12.5
t_wait = 240
n = int(((T_Stop-T_Start)/T_Step)+1)

wavelength_start = 1565
wavelength_stop = 1585
wavelength_points = 2001
OSA_Resolution = 0.03
OSA_VBW = 1000
timeout = 10000

bias_start = 0
bias_stop = 500
bias_step = 2
bias_current = np.linspace(bias_start,bias_stop,int((bias_stop-bias_start)/bias_step) + 1)

Light_engine = "ELS 24-12-0001"
Channel = 2

osa.set_timeout(20000)

#### Function definition

In [47]:
def sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW):
    osa.set_wavelength(wavelength_start, wavelength_stop, wavelength_points)
    osa.set_resolution_VBW(OSA_Resolution,OSA_VBW)
    osa.sweep_single()
    Power_list = osa.get_sweep_result()
    Power_list_uW = [None] * len(Power_list)
    for i in range(0,len(Power_list)):
        Power_list_uW[i] = 10**(float(Power_list[i])/10)*1000 
    Wavelength_list = np.linspace(wavelength_start,wavelength_stop,wavelength_points)
    Wavelength_peak_nm, Power_peak_dBm = osa.get_peak()   
    if Power_peak_dBm >= -30:
        SMSR_Linewidth_nm, SMSR_dB = osa.get_SMSR()
        Linewidth_3db_nm = osa.get_linewidth(3)
        Linewidth_20db_nm = osa.get_linewidth(20)
    else:
        SMSR_Linewidth_nm, SMSR_dB = 0, 0
        Linewidth_3db_nm = 0
        Linewidth_20db_nm = 0
    return  Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm

## Measurement routine

In [48]:
meas_df = pd.DataFrame()

In [49]:
for k in range(0,n,1):
    rf = le.set_laser_current(Channel,0)
    T_Set = T_Start+k*T_Step
    rf = le.set_temperature(T_Set)
    print(rf)
    rf = le.set_tec_output_on()
    print(rf)
    time.sleep(t_wait)
    rf = le.measure_temperature()
    print(rf)
    rf = le.set_output_on(Channel)
    print(rf)
    for j in range(0,int(((bias_stop-bias_start)/bias_step)+1),bias_step):     
        rf = le.set_laser_current(Channel,bias_current[j])
        T_read = le.measure_temperature()
        Voltage_V = le.measure_laser_voltage(Channel)
        bias_current_meas = le.measure_laser_current(Channel)
        Wavelength_list, Power_list, Power_list_uW, Wavelength_peak_nm, Power_peak_dBm, SMSR_Linewidth_nm, SMSR_dB, Linewidth_3db_nm, Linewidth_20db_nm = sweepOSA(osa, wavelength_start, wavelength_stop, wavelength_points, OSA_Resolution, OSA_VBW)
        meas_df_temp = pd.DataFrame({'Light_engine':Light_engine, 'Channel':Channel, 'Bias_current_mA':bias_current[j], 'Bias_current_meas_mA':bias_current_meas,'Voltage_V':Voltage_V, 'Temeprature_C':T_read, 'Wavelength_nm':Wavelength_list, 'Power_dBm':Power_list, 
                                    'Power_uW':Power_list_uW, 'Wavelength_peak_nm':Wavelength_peak_nm, 'Power_peak_nm':Power_peak_dBm, 'SMSR_dB':SMSR_dB, 'SMSR_linewidth_nm':SMSR_Linewidth_nm,
                                    '3dB_linewidth_nm':Linewidth_3db_nm, '20dB_linewidth_nm':Linewidth_20db_nm})
        data = [meas_df_temp, meas_df]
        meas_df = pd.concat(data,ignore_index=True,sort=False)

Set TEC target temperature 25.00
Set TEC ON
24.97
Set channel 2 ON
Set TEC target temperature 37.50
Set TEC ON
37.53
Set channel 2 ON
Set TEC target temperature 50.00
Set TEC ON
50.05
Set channel 2 ON


## Save data and reset values

In [50]:
rf = le.set_tec_output_off()
print(rf)
rf = le.set_temperature(25)
print(rf)
rf = le.set_output_off(Channel)
print(rf)
rf = le.set_laser_current(Channel,0)
print(rf)

Set TEC OFF
Set TEC target temperature 25.00
Set channel 2 OFF
Set Current value 0.00 to channel :2


In [51]:
meas_df.to_csv(r"ELS-24-12-0001_Channel_2.csv")

In [52]:
meas_df

,Light_engine,Channel,Bias_current_mA,Bias_current_meas_mA,Voltage_V,Temeprature_C,Wavelength_nm,Power_dBm,Power_uW,Wavelength_peak_nm,Power_peak_nm,SMSR_dB,SMSR_linewidth_nm,3dB_linewidth_nm,20dB_linewidth_nm
0,ELS 24-12-0001,2,500.0,495.22,2.04,50.09,1565.00,-49.46,0.011324,1581.44,16.22,56.23,0.48,0.082,0.029
1,ELS 24-12-0001,2,500.0,495.22,2.04,50.09,1565.01,-50.02,0.009954,1581.44,16.22,56.23,0.48,0.082,0.029
2,ELS 24-12-0001,2,500.0,495.22,2.04,50.09,1565.02,-49.24,0.011912,1581.44,16.22,56.23,0.48,0.082,0.029
3,ELS 24-12-0001,2,500.0,495.22,2.04,50.09,1565.03,-49.34,0.011641,1581.44,16.22,56.23,0.48,0.082,0.029
4,ELS 24-12-0001,2,500.0,495.22,2.04,50.09,1565.04,-49.18,0.012078,1581.44,16.22,56.23,0.48,0.082,0.029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756373,ELS 24-12-0001,2,0.0,9.48,0.00,24.94,1584.96,-56.79,0.002094,1576.43,-49.47,0.00,0.00,0.000,0.000
756374,ELS 24-12-0001,2,0.0,9.48,0.00,24.94,1584.97,-55.07,0.003112,1576.43,-49.47,0.00,0.00,0.000,0.000
756375,ELS 24-12-0001,2,0.0,9.48,0.00,24.94,1584.98,-58.55,0.001396,1576.43,-49.47,0.00,0.00,0.000,0.000
756376,ELS 24-12-0001,2,0.0,9.48,0.00,24.94,1584.99,-54.28,0.003733,1576.43,-49.47,0.00,0.00,0.000,0.000


## Clear handles

In [55]:
le.close()
osa.close()